read the data from nist... file   
    problem may come up: can't read the data    
                         can't related the data with colunm     
                         don't know the unit(??/pixel)    
                         type error     
   
   
use plt.hist    
    
mapping wafer    
    pro: how to mapping?    
         how to do sth like circle backgrond     
         




In [26]:
#This will be the program first use histogram  and then mapping the wafer

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import pandas as pd

#Read the data
data = pd.read_csv('nist_so_mf_detector_array_padinfo.csv')

#get columns vaule about pixel positon
pixel_x_center = data['Pixel X center']
pixel_y_center = data['Pixel Y center']
#remove the column name
pixel_x_center = pixel_x_center[1:]
pixel_y_center = pixel_y_center[1:]

#histogram
plt.hist2d(pixel_x_center, pixel_y_center, bins = (1000,1000), norm=colors.LogNorm())
#colors.LogNorm() normalize a given value to the 0-1 range on a log scale to show the log scale
# use norm to emphasize the exist value 
plt.colorbar()
plt.show()

#get interesting plot I was excepted a histogram 
# but I forget to check the pair of wafer and just use a hist2d to get this plot
# this at least show the wafer location in the plot

In [ ]:
#need to group the pair of wafer
#Don't know the pair of wafer

In [ ]:
#try to get the unit for per pixel from the length of the focal plane
#   
sizeFocalPlane = 150 #dm = 1.5m
